In [1]:
import os

In [2]:
%pwd

'c:\\Users\\LENOVO\\Animal_Classifier\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\LENOVO\\Animal_Classifier'

In [ ]:
# Imports and Setup
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

### 📂 Load and Preprocess Dataset
We load the images from the `dataset/` folder using TensorFlow's `image_dataset_from_directory`.
We then normalize the pixel values.


In [ ]:
#  Load Dataset
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
DATASET_PATH = "dataset/"

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH, validation_split=0.2, subset="training", seed=123,
    image_size=IMG_SIZE, batch_size=BATCH_SIZE)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH, validation_split=0.2, subset="validation", seed=123,
    image_size=IMG_SIZE, batch_size=BATCH_SIZE)

# Normalize the images
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))


Found 1944 files belonging to 15 classes.
Using 1556 files for training.
Found 1944 files belonging to 15 classes.
Using 388 files for validation.


### 🧠 Define the Model (Transfer Learning)
We use MobileNetV2 as a base model and build a classifier on top of it.

In [ ]:
# Define Model
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(15, activation='softmax')
])


### 🏋️‍♂️ Train the Model in Two Phases
- Phase 1: Train only the top layers.
- Phase 2: Unfreeze and fine-tune the base model.

In [9]:
#' Phase 1 Training (frozen base model)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history_1 = model.fit(train_dataset, validation_data=val_dataset, epochs=5)

Epoch 1/5


49/49 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - accuracy: 0.9466 - loss: 0.1665 - val_accuracy: 0.8943 - val_loss: 0.3185
Epoch 2/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.9663 - loss: 0.1035 - val_accuracy: 0.8995 - val_loss: 0.3015
Epoch 3/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.9781 - loss: 0.0799 - val_accuracy: 0.9149 - val_loss: 0.2777
Epoch 4/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 74s 2s/step - accuracy: 0.9826 - loss: 0.0587 - val_accuracy: 0.8969 - val_loss: 0.3256
Epoch 5/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.9871 - loss: 0.0491 - val_accuracy: 0.8969 - val_loss: 0.3163


In [10]:
# Phase 2 Training (fine-tune base model)
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit(train_dataset, validation_data=val_dataset, epochs=5)

Epoch 1/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 384s 7s/step - accuracy: 0.8327 - loss: 0.5825 - val_accuracy: 0.9046 - val_loss: 0.2834
Epoch 2/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 363s 7s/step - accuracy: 0.9107 - loss: 0.2604 - val_accuracy: 0.9175 - val_loss: 0.2484
Epoch 3/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 330s 6s/step - accuracy: 0.9351 - loss: 0.2165 - val_accuracy: 0.9227 - val_loss: 0.2236
Epoch 4/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 293s 6s/step - accuracy: 0.9595 - loss: 0.1347 - val_accuracy: 0.9253 - val_loss: 0.2072
Epoch 5/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 278s 6s/step - accuracy: 0.9549 - loss: 0.1391 - val_accuracy: 0.9330 - val_loss: 0.1943


### 💾 Save the Trained Model


In [11]:
# Save Model
os.makedirs("models", exist_ok=True)
model.save("models/cnn_model.keras")

### 📊 Plot Accuracy
Visualize how the model's accuracy evolved over the training epochs.


In [13]:
# Plot Accuracy
train_acc = history_1.history['accuracy'] + history_2.history['accuracy']
val_acc = history_1.history['val_accuracy'] + history_2.history['val_accuracy']

plt.figure(figsize=(10, 5))
plt.plot(train_acc, label="Train Accuracy")
plt.plot(val_acc, label="Validation Accuracy")
plt.legend()
plt.title("Model Accuracy Over Epochs")
plt.savefig("training_accuracy_plot.png", dpi=300)
plt.close()